In [1]:
from app.agent.multi_agent import MultiAgent
from app.agent.agent import ContextTypeAgent, QuestionTypeAgent
from app.prompts.prompts import SIMPLE_VS_COMPLEX, GENERAL_VS_PARTICULAR_CONTEXT
from app.prompts.prompt import Prompt

import asyncio
import nest_asyncio
nest_asyncio.apply()

# simple vs complex -> 0-1 vs > 1 subjects
simple_vs_complex_prompt = Prompt(prompt=SIMPLE_VS_COMPLEX)
simple_vs_complex_agent = QuestionTypeAgent(
    instruction=simple_vs_complex_prompt
)

# general vs particular
general_vs_particular_prompt = Prompt(GENERAL_VS_PARTICULAR_CONTEXT)
general_vs_particular_agent = ContextTypeAgent(
    instruction=general_vs_particular_prompt
    )

multi_agent = MultiAgent(agents=[
    simple_vs_complex_agent, 
    general_vs_particular_agent
])

/Users/Jaime/Desktop/Project-Ideas/python-gpt/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from app.database.base import get_db
from sqlalchemy.orm import Session

db = next(get_db())

In [3]:
from app.retrievers.general_retriever import GeneralRetriever
from app.retrievers.similarity_retriever import SimilarityRetriever


async def tool_pipeline(agent: MultiAgent, query: str, db: Session=db):
    tool, output = await agent.pipeline(query=query)
    tool: GeneralRetriever | SimilarityRetriever = tool(db=db)
    return tool.query_database(query=query, subjects=output.subject)

In [6]:
from app.agent.llama_client import LlamaClient
from app.printer import Printer
from app.prompts.prompts import PROMPT_TO_ANSWER_QUESTIONS

printer = Printer()
query = "How does the function _create_file_node depend on the function _create_nodes_of_file?"

llm = LlamaClient()

def format_answer(answer: str, max_words: int) -> str:
    answer = answer.replace("\n\n", "\n")
    lines = answer.split("\n")
    processed_answer = " "
    for line in lines:
        words = line.split(" ")
        for k in range(0, len(words), max_words):
            processed_line = " ".join(words[k: k + max_words])
            processed_answer += "\n" + processed_line
    return processed_answer

async def query_pipeline(agent: MultiAgent, 
                         query: str, 
                         llm: LlamaClient, 
                         db: Session) -> str:
    
    tool, output = await agent.pipeline(query=query)
    printer.print_blue(f"Final subjects: {output.subject}")
    tool: GeneralRetriever | SimilarityRetriever = tool(db=db)
    nodes, nodes_with_score, relationships = tool.query_database(query=query, subjects=output.subject)
    if nodes_with_score is None and relationships is None:
        printer.print_blue(f"We are on a general case with 0 subjects. Asking the LLM one node at a time...")
        answers = []
        for node in nodes[:15]:
            context = node.text
            prompt = Prompt.format_prompt(prompt=PROMPT_TO_ANSWER_QUESTIONS, context=context, query=query)
            answer = await llm.acall(query=prompt)
            answers.append((answer, node.text[:100], node.id))
        return answers, None, None
    
    for node_with_score in nodes_with_score:
        printer.print_blue(f"Score: {node_with_score.score} for text: \n{node_with_score.node.text[:300]}\n")
    if relationships: 
        for relation, relation_nodes in relationships.items():
            for n, rel_node in enumerate(relation_nodes):
                printer.print_blue(f"\tRelationship {n+1} for node --> {relation}: \n{rel_node.text[:150]}\n")
    context = "\n".join([node.text for node in nodes])
    prompt = Prompt.format_prompt(prompt=PROMPT_TO_ANSWER_QUESTIONS, context=context, query=query)
    answer = await llm.acall(query=prompt)
    return answer, nodes, relationships

In [7]:
query = "Are there any errors in the repository?"
answer, nodes, relationships = asyncio.run(
    query_pipeline(agent=multi_agent, 
                   query=query, 
                   llm=llm, 
                   db=db)
)

Agent reasoning response: There is no explicit subject mentioned in the question, therefore it is considered simple since there are no subjects.
Agent answer: simple 

Bad parsing from the llm. Trying again...
Agent reasoning response: The question does not refer only to a specific subject, therefore the answer is general
Agent answer: general 

Tool decided by the agent: GeneralRetriever
Final subjects: set()


/Users/Jaime/Desktop/Project-Ideas/python-gpt/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


We are on a general case with 0 subjects. Ask the LLM one by one...


**Let's take a look at only the first 15 nodes (hardcoded in the function query_pipeline)**

In [10]:
for (ans, text, node_id) in answer:
    print(node_id)
    print(text)
    print(format_answer(ans, max_words=15))
    print("-"*200)

7a1509e8-abef-408a-8ab4-0d8fd67ecb86
async def main_page():
  return {'msg': 'welcome to the main page!'}

 
I'm happy to help!
Looking at the provided code, I don't see any syntax errors or obvious issues that
would cause the code to not work as intended.
The `main_ page()` function is defined with the correct indentation and uses valid Python syntax.
The return statement returns a dictionary with a single key-value pair, which appears to be
a valid JSON response.
However, without more context about the repository or how this code is being used, it's
difficult for me to say definitively whether there are any errors in the repository or
not. 
To better answer your question, could you please provide more information about the repository, such
as:
* What kind of project is this (e.g., web app, script, etc.)?
* How is the code being used (e.g., is it part of a larger framework,
or is it standalone)?
* Are there any specific errors or issues you're experiencing?
With more context, I'd be h